In [3]:
import pandas as pd
from itertools import combinations

data = pd.read_csv("../data/data_to_process.csv")
print(len(data))
data.head()

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0
3,2019,6,Is amygdalin or laetrile an effective cancer t...,amygdalin laetrile cancer,0.0
4,2019,7,Can aspirin improve the lives of people with v...,aspirin vascular dementia,0.0


## Keywords

In [25]:
biomed_explorer_data = pd.read_csv("../data/biomed_explorer/biomed_explorer_abstracts_keywords.csv")
pubmed_search_data = pd.read_csv("../data/pubmed_search/pubmed_search_abstracts_keywords.csv").dropna()

google_search_data = pd.concat(
    (
        pd.read_csv("../data/google_search/google_search_abstracts_keywords_part_1.csv"), 
        pd.read_csv("../data/google_search/google_search_abstracts_keywords_part_2.csv")
    )
).astype({"query_id": str})

wikipedia_search_wiki_data = pd.read_csv("../data/wikipedia_search_wiki/wikipedia_articles_keywords.csv")
google_search_wiki_data = pd.read_csv("../data/google_search_wiki/wikipedia_articles_keywords.csv")

dct = {
    "pubmed_search": pubmed_search_data,
    "google_search": google_search_data,
    "biomed_explorer": biomed_explorer_data,
    "wikipedia_search_wiki": wikipedia_search_wiki_data,
    "google_search_wiki": google_search_wiki_data,
}

In [26]:
data_source2failed_queries = dict()

for data_source, received_data in dct.items():
    failed_queries = data.merge(
                        received_data.drop_duplicates(['query_id', "data_source"]),
                        on=['query_id', 'data_source'],
                        how='left',
                        indicator=True
                    ).query("_merge == 'left_only'")
    
    if "query" in failed_queries.columns:
        failed_queries = failed_queries['query']
    else:
        failed_queries = failed_queries['query_x']
    
    failed_queries = {q.strip() for q in failed_queries}
    data_source2failed_queries[data_source] = failed_queries

## Question

In [27]:
biomed_explorer_data = pd.read_csv("../data/biomed_explorer/biomed_explorer_abstracts_question.csv")
pubmed_search_data = pd.read_csv("../data/pubmed_search/pubmed_search_abstracts_question.csv").dropna()

google_search_data = pd.concat(
    (
        pd.read_csv("../data/google_search/google_search_abstracts_question_part_1.csv"), 
        pd.read_csv("../data/google_search/google_search_abstracts_question_part_2.csv")
    )
).astype({"query_id": str})

wikipedia_search_wiki_data = pd.read_csv("../data/wikipedia_search_wiki/wikipedia_articles_question.csv")
google_search_wiki_data = pd.read_csv("../data/google_search_wiki/wikipedia_articles_question.csv")

dct = {
    "pubmed_search": pubmed_search_data,
    "google_search": google_search_data,
    "biomed_explorer": biomed_explorer_data,
    "wikipedia_search_wiki": wikipedia_search_wiki_data,
    "google_search_wiki": google_search_wiki_data,
}

In [28]:
data_source2failed_descriptions = dict()

for data_source, received_data in dct.items():
    failed_descriptions = data.merge(
                        received_data.drop_duplicates(['query_id', "data_source"]),
                        on=['query_id', 'data_source'],
                        how='left',
                        indicator=True
                    ).query("_merge == 'left_only'")
    
    if "query" in failed_descriptions.columns:
        failed_descriptions = failed_descriptions['query']
    else:
        failed_descriptions = failed_descriptions['query_x']
        
    failed_descriptions = {q.strip() for q in failed_descriptions}
    data_source2failed_descriptions[data_source] = failed_descriptions

In [29]:
data_source2failed_keywords = {source: len(queries) for source, queries in data_source2failed_queries.items()}
data_source2failed_question = {source: len(queries) for source, queries in data_source2failed_descriptions.items()}

In [32]:
pd.DataFrame(
    data={
        "Keywords": data_source2failed_keywords.values(), 
        "Question": data_source2failed_question.values()
    },
    index=dct.keys()
)

,Keywords,Question
pubmed_search,31,42
google_search,1,2
biomed_explorer,0,0
wikipedia_search_wiki,26,56
google_search_wiki,19,16
